# STEP 2 - Evaluation of the Biggest Model

With a simple central model found, it is now time to evaluate the prediction quality of a model incorporating all available features.
This model is expected to provide a similar if not worse prediction quality than the first central model.
Reason for that expectation is the fact that features like longitude, latitude and the venue_id could confuse a model.
The relation between these features and the target feature is not recognizable from the data alone (i.e. physical coordinates to semantic place).

## Imports

In [1]:
import nest_asyncio

nest_asyncio.apply()

import collections
import functools
import os
import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [2]:
import logging

logging.basicConfig(filename="./log/biggest-model/Evaluation.log", level=logging.INFO)

def log(text):
  print(text)
  logging.info(text)

In [3]:
# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Biggest Model

This model incorporates the maximum amount of available and useful data.

In [4]:
df = pd.read_csv("./4square/processed_transformed_big.csv")
df.head(100)

,cat_id,user_id,latitude,longitude,is_weekend,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos,venue_id,orig_cat_id
0,0,470,40.719810,-74.002581,False,-1.000000,0.000654,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,0,0
1,1,979,40.606800,-74.044170,False,-0.999998,0.001818,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,1,1
2,2,69,40.716162,-73.883070,False,-0.999945,0.010472,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,2,2
3,3,395,40.745164,-73.982519,False,-0.999931,0.011708,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,3,3
4,4,87,40.740104,-73.989658,False,-0.999914,0.013090,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,445,40.828602,-73.879259,False,-0.959601,0.281365,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,93,24
96,6,235,40.745463,-73.990983,False,-0.956326,0.292302,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,94,6
97,8,118,40.600144,-73.946593,False,-0.955729,0.294249,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,95,57
98,2,1054,40.870630,-74.097926,False,-0.955407,0.295291,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349,96,58


It is best, to use only the best 100 users for this purpose.
As they have the longest sequences of visited places.

In [5]:
count = df.user_id.value_counts()

idx = count.loc[count.index[:100]].index # count >= 100
df = df.loc[df.user_id.isin(idx)]

An array is created containing all visited locations for every user.
The original data is sorted by time (ascending).
Thus, the array contains a sequence of visited location categories by user.

In [6]:
# List the df for each user
users_locations = []

# For each user
for user_id in tqdm(idx):
  users_locations.append(df.loc[df.user_id == user_id].copy())

100%|██████████| 100/100 [00:00<00:00, 2222.21it/s]


It is necessary to first split the data in train, valid and test for each user.
Then, these are merged together again later on.
This is done to ensure that the sequences are kept together and not split randomly for the users.

In [7]:
# List the dfs fo train, val and test for each user
users_locations_train = []
users_locations_val = []
users_locations_test = []

for user_df in users_locations:
  # Split in train, test and validation
  train, test = train_test_split(user_df, test_size=0.2, shuffle=False)
  train, val = train_test_split(train, test_size=0.2, shuffle=False)

  # Append the sets
  users_locations_train.append(train)
  users_locations_val.append(val)
  users_locations_test.append(test)

The dataframes are concatenated again.

In [8]:
# Merge back the dataframes
df_train = pd.concat(users_locations_train)

# Merge back the dataframes
df_val = pd.concat(users_locations_val)

# Merge back the dataframes
df_test = pd.concat(users_locations_test)

Sanity check: Was data lost when splitting and merging back together?

In [9]:
user_ids = df_train.user_id.unique()
print(user_ids.size)

100


No, everything is fine.
Now, the data has to be split in sequences of length N.
The following code is structured in methods, so the best value for N can be found.

Get the column names for the method below.

In [10]:
columns_names = df_train.columns.values
print(columns_names)

['cat_id' 'user_id' 'latitude' 'longitude' 'is_weekend' 'clock_sin'
 'clock_cos' 'day_sin' 'day_cos' 'month_sin' 'month_cos' 'week_day_sin'
 'week_day_cos' 'venue_id' 'orig_cat_id']


In [11]:
# List of numerical column names
numerical_column_names = ['latitude', 'longitude', 'clock_sin', 'clock_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'week_day_sin', 'week_day_cos']

In [12]:
class CategoricalFeature:
  def __init__(self, feature_name, vocab_size, use_embedding):
    self.feature_name = feature_name
    self.vocab_size = vocab_size
    self.use_embedding = use_embedding

In [13]:
vocab_size = df.cat_id.unique().size
users_size = df.user_id.unique().size
venues_size = df.venue_id.unique().size
orig_cats_size = df.orig_cat_id.unique().size

print('Unique uber categories: ', vocab_size)
print('Unique users: ', users_size)
print('Unique venues: ', venues_size)
print('Unique original categories: ', orig_cats_size)

Unique uber categories:  27
Unique users:  100
Unique venues:  11331
Unique original categories:  368


In [14]:
categorical_columns = [
  CategoricalFeature('user_id', users_size, False),
  CategoricalFeature('cat_id', vocab_size, False),
  CategoricalFeature('venue_id', venues_size, False),
  CategoricalFeature('orig_cat_id', orig_cats_size, False)]

Tunable Parameters:

In [15]:
NUM_CLIENTS = user_ids.size
NUM_EPOCHS = 4
BATCH_SIZE = 16
#SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 5

Helper functions to split data, create clients dictionaries and preprocess the data for the FL algorithm.
The model creation is also defined here.

In [16]:
# Split the data into chunks of N
def split_data(N):

  # dictionary of list of df
  df_dictionary = {}

  for uid in tqdm(user_ids):
    # Get the records of the user
    user_df_train = df_train.loc[df_train.user_id == uid].copy()
    user_df_val = df_val.loc[df_val.user_id == uid].copy()
    user_df_test = df_test.loc[df_test.user_id == uid].copy()

    # Get a list of dataframes of length N records
    user_list_train = [user_df_train[i:i+N] for i in range(0, user_df_train.shape[0], N)]
    user_list_val = [user_df_val[i:i+N] for i in range(0, user_df_val.shape[0], N)]
    user_list_test = [user_df_test[i:i+N] for i in range(0, user_df_test.shape[0], N)]

    # Save the list of dataframes into a dictionary
    df_dictionary[uid] = {
        'train': user_list_train,
        'val': user_list_val,
        'test': user_list_test
    }

  return  df_dictionary

In [17]:
# Takes a dictionary with train, validation and test sets and the desired set type
def create_clients_dict(df_dictionary, set_type, N):

  dataset_dict = {}

  for uid in tqdm(user_ids):

    c_data = collections.OrderedDict()
    values = df_dictionary[uid][set_type]

    # If the last dataframe of the list is not complete
    if len(values[-1]) < N:
      diff = 1
    else:
      diff = 0

    if len(values) > 0:
      # Create the dictionary to create a clientData
      for header in columns_names:
        c_data[header] = [values[i][header].values for i in range(0, len(values)-diff)]
      dataset_dict[uid] = c_data

  return dataset_dict

In [18]:
# preprocess dataset to tf format
def preprocess(dataset, N):

  def batch_format_fn(element):

    x=collections.OrderedDict()

    for name in columns_names:
      x[name]=tf.reshape(element[name][:, :-1], [-1, N-1])

    y=tf.reshape(element[columns_names[0]][:, 1:], [-1, N-1])

    return collections.OrderedDict(x=x, y=y)

  return dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [19]:
# create federated data for every client
def make_federated_data(client_data, client_ids, N):

  return [
      preprocess(client_data.create_tf_dataset_for_client(x), N)
      for x in tqdm(client_ids)
  ]

In [20]:
# Create a model
def create_keras_model(number_of_places, N, batch_size):

  # Shortcut to the layers package
  l = tf.keras.layers

  # List of numeric feature columns to pass to the DenseLayer
  numeric_feature_columns = []

  # Handling numerical columns
  for header in numerical_column_names:
		# Append all the numerical columns defined into the list
    numeric_feature_columns.append(feature_column.numeric_column(header, shape=N-1))

  feature_inputs={}
  for c_name in numerical_column_names:
    feature_inputs[c_name] = tf.keras.Input((N-1,), batch_size=batch_size, name=c_name)

  # We cannot use an array of features as always because we have sequences
  # We have to do one by one in order to match the shape
  num_features = []
  for c_name in numerical_column_names:
    f =  feature_column.numeric_column(c_name, shape=(N-1))
    feature = l.DenseFeatures(f)(feature_inputs)
    feature = tf.expand_dims(feature, -1)
    num_features.append(feature)

  categorical_feature_inputs = []
  categorical_features = []
  for categorical_feature in categorical_columns:  # add batch_size=batch_size in case of stateful GRU
    d = {categorical_feature.feature_name: tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name=categorical_feature.feature_name)}
    categorical_feature_inputs.append(d)

    one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(categorical_feature.feature_name, [i for i in range(categorical_feature.vocab_size)])

    if categorical_feature.use_embedding:
      # Embed the one-hot encoding
      categorical_features.append(feature_column.embedding_column(one_hot, 256))
    else:
      categorical_features.append(feature_column.indicator_column(one_hot))

  seq_features = []
  for i in range(0, len(categorical_feature_inputs)):
    sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(categorical_features[i])(categorical_feature_inputs[i])
    seq_features.append(sequence_features)

  input_sequence = l.Concatenate(axis=2)( [] + seq_features + num_features)

  # Rnn
  recurrent = l.GRU(256,
                        batch_size=batch_size, #in case of stateful
                        dropout=0.3,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(input_sequence)


	# Last layer with an output for each place
  dense_1 = layers.Dense(number_of_places)(recurrent)

	# Softmax output layer
  output = l.Softmax()(dense_1)

	# To return the Model, we need to define its inputs and outputs
	# In out case, we need to list all the input layers we have defined
  inputs = list(feature_inputs.values()) + categorical_feature_inputs

	# Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [21]:
#train and evaluate the model
def train_and_eval_model(vocab_size, n, federated_train_data, federated_val_data, federated_test_data, path='./log/central-test-run'):
  train_logdir = path + '/train'
  val_logdir = path + '/val'
  eval_logdir = path + '/eval'

  train_summary_writer = tf.summary.create_file_writer(train_logdir)
  val_summary_writer = tf.summary.create_file_writer(val_logdir)
  eval_summary_writer = tf.summary.create_file_writer(eval_logdir)

  # Clone the keras_model inside `create_tff_model()`, which TFF will
  # call to produce a new copy of the model inside the graph that it will
  # serialize. Note: we want to construct all the necessary objects we'll need
  # _inside_ this method.
  def create_tff_model():
    # TFF uses an `input_spec` so it knows the types and shapes
    # that your model expects.
    input_spec = federated_train_data[0].element_spec
    keras_model_clone = create_keras_model(vocab_size, n, batch_size=BATCH_SIZE)
    #plot_model(keras_model_clone, 'keras_model_for_fl.png', show_shapes=True)
    tff_model = tff.learning.from_keras_model(
      keras_model_clone,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return tff_model

  # This command builds all the TensorFlow graphs and serializes them:
  fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.002),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.06))

  state = fed_avg.initialize()
  evaluation = tff.learning.build_federated_evaluation(model_fn=create_tff_model)

  tolerance = 7
  best_state = 0
  lowest_loss = 100.00
  stop = tolerance

  NUM_ROUNDS = 10
  with train_summary_writer.as_default():
    for round_num in range(1, NUM_ROUNDS + 1):
      log('Round {r}'.format(r=round_num))

      # Uncomment to simulate sparse availability of clients
      # train_data_for_this_round, val_data_for_this_round = sample((federated_train_data, federated_val_data), 20, NUM_CLIENTS)

      state, metrics = fed_avg.next(state, federated_train_data)

      train_metrics = metrics['train']
      log('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(l=train_metrics['loss'], a=train_metrics['sparse_categorical_accuracy']))

      val_metrics = evaluation(state.model, federated_val_data)
      log('\tValidation: loss={l:.3f}, accuracy={a:.3f}'.format( l=val_metrics['loss'], a=val_metrics['sparse_categorical_accuracy']))

      # Check for decreasing validation loss
      if lowest_loss > val_metrics['loss']:
        log('\tSaving best model..')
        lowest_loss = val_metrics['loss']
        best_state = state
        stop = tolerance - 1
      else:
        stop = stop - 1
        if stop <= 0:
          log('\tEarly stopping...')
          break;

      log(' ')
      log('\twriting..')

      # Iterate across the metrics and write their data
      for name, value in dict(train_metrics).items():
        tf.summary.scalar('epoch_'+name, value, step=round_num)

      with val_summary_writer.as_default():
        for name, value in dict(val_metrics).items():
          tf.summary.scalar('epoch_'+name, value, step=round_num)

  train_summary_writer.close()
  val_summary_writer.close()

  # evaluate over test data
  test_metrics = evaluation(best_state.model, federated_test_data)
  log('\tEvaluation: loss={l:.3f}, accuracy={a:.3f}'.format( l=test_metrics['loss'], a=test_metrics['sparse_categorical_accuracy']))

First, we do a test run for N=16 and see if everything works.
Then, we run the same logic for different lengths of sequences and compare the results.

In [22]:
n=17
df_dict = split_data(n)
clients_train_dict = create_clients_dict(df_dict, 'train', n)
clients_val_dict = create_clients_dict(df_dict, 'val', n)
clients_test_dict = create_clients_dict(df_dict, 'test', n)

100%|██████████| 100/100 [00:00<00:00, 398.41it/s]


In [23]:
# Convert the dictionary to a dataset
client_train_data = tff.simulation.FromTensorSlicesClientData(clients_train_dict)
client_val_data = tff.simulation.FromTensorSlicesClientData(clients_val_dict)
client_test_data = tff.simulation.FromTensorSlicesClientData(clients_test_dict)

In [24]:
client_train_data.create_tf_dataset_for_client(user_ids[0]).element_spec

OrderedDict([('cat_id', TensorSpec(shape=(17,), dtype=tf.int32, name=None)),
             ('user_id', TensorSpec(shape=(17,), dtype=tf.int32, name=None)),
             ('latitude',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('longitude',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('is_weekend', TensorSpec(shape=(17,), dtype=tf.bool, name=None)),
             ('clock_sin',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('clock_cos',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('day_sin', TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('day_cos', TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('month_sin',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('month_cos',
              TensorSpec(shape=(17,), dtype=tf.float64, name=None)),
             ('week_day_sin',


In [25]:
example_dataset = client_train_data.create_tf_dataset_for_client(
    client_train_data.client_ids[1])

example_element = next(iter(example_dataset))
example_element

OrderedDict([('cat_id',
              <tf.Tensor: shape=(17,), dtype=int32, numpy=array([ 9, 11, 22, 21,  9, 16, 18,  1,  7, 11,  1, 11, 21, 16, 25, 22, 18])>),
             ('user_id',
              <tf.Tensor: shape=(17,), dtype=int32, numpy=
              array([185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185, 185,
                     185, 185, 185, 185])>),
             ('latitude',
              <tf.Tensor: shape=(17,), dtype=float64, numpy=
              array([40.96541694, 40.96532878, 40.96535502, 40.96533795, 40.96535583,
                     40.96532194, 40.96541698, 40.96535005, 40.9645718 , 40.96532878,
                     40.96535005, 40.96536873, 40.96533795, 40.96532194, 40.96550875,
                     40.96535502, 40.96541698])>),
             ('longitude',
              <tf.Tensor: shape=(17,), dtype=float64, numpy=
              array([-74.06288376, -74.06292392, -74.06283213, -74.0628079 ,
                     -74.06285508, -74.06280345, -74.06291968, 

After looking at an example dataset, it can be concluded that the layout of the data is also as expected.

In [26]:
preprocessed_example_dataset = preprocess(example_dataset, n)
sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch['x']['cat_id'].shape

(16, 16)

In [27]:
preprocessed_example_dataset

<PrefetchDataset shapes: OrderedDict([(x, OrderedDict([(cat_id, (16, 16)), (user_id, (16, 16)), (latitude, (16, 16)), (longitude, (16, 16)), (is_weekend, (16, 16)), (clock_sin, (16, 16)), (clock_cos, (16, 16)), (day_sin, (16, 16)), (day_cos, (16, 16)), (month_sin, (16, 16)), (month_cos, (16, 16)), (week_day_sin, (16, 16)), (week_day_cos, (16, 16)), (venue_id, (16, 16)), (orig_cat_id, (16, 16))])), (y, (16, 16))]), types: OrderedDict([(x, OrderedDict([(cat_id, tf.int32), (user_id, tf.int32), (latitude, tf.float64), (longitude, tf.float64), (is_weekend, tf.bool), (clock_sin, tf.float64), (clock_cos, tf.float64), (day_sin, tf.float64), (day_cos, tf.float64), (month_sin, tf.float64), (month_cos, tf.float64), (week_day_sin, tf.float64), (week_day_cos, tf.float64), (venue_id, tf.int32), (orig_cat_id, tf.int32)])), (y, tf.int32)])>

In [28]:
preprocessed_example_dataset.element_spec

OrderedDict([('x',
              OrderedDict([('cat_id',
                            TensorSpec(shape=(16, 16), dtype=tf.int32, name=None)),
                           ('user_id',
                            TensorSpec(shape=(16, 16), dtype=tf.int32, name=None)),
                           ('latitude',
                            TensorSpec(shape=(16, 16), dtype=tf.float64, name=None)),
                           ('longitude',
                            TensorSpec(shape=(16, 16), dtype=tf.float64, name=None)),
                           ('is_weekend',
                            TensorSpec(shape=(16, 16), dtype=tf.bool, name=None)),
                           ('clock_sin',
                            TensorSpec(shape=(16, 16), dtype=tf.float64, name=None)),
                           ('clock_cos',
                            TensorSpec(shape=(16, 16), dtype=tf.float64, name=None)),
                           ('day_sin',
                            TensorSpec(shape=(16, 16), dtype=tf.f

In [29]:
# Select the clients
sample_clients = client_train_data.client_ids[0:NUM_CLIENTS]

# Federate the clients datasets
federated_train_data = make_federated_data(client_train_data, sample_clients, n)
federated_val_data = make_federated_data(client_val_data, sample_clients, n)
federated_test_data = make_federated_data(client_test_data, sample_clients, n)

print('\nNumber of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

100%|██████████| 100/100 [00:03<00:00, 27.50it/s]


Number of client datasets: 100
First dataset: <PrefetchDataset shapes: OrderedDict([(x, OrderedDict([(cat_id, (16, 16)), (user_id, (16, 16)), (latitude, (16, 16)), (longitude, (16, 16)), (is_weekend, (16, 16)), (clock_sin, (16, 16)), (clock_cos, (16, 16)), (day_sin, (16, 16)), (day_cos, (16, 16)), (month_sin, (16, 16)), (month_cos, (16, 16)), (week_day_sin, (16, 16)), (week_day_cos, (16, 16)), (venue_id, (16, 16)), (orig_cat_id, (16, 16))])), (y, (16, 16))]), types: OrderedDict([(x, OrderedDict([(cat_id, tf.int32), (user_id, tf.int32), (latitude, tf.float64), (longitude, tf.float64), (is_weekend, tf.bool), (clock_sin, tf.float64), (clock_cos, tf.float64), (day_sin, tf.float64), (day_cos, tf.float64), (month_sin, tf.float64), (month_cos, tf.float64), (week_day_sin, tf.float64), (week_day_cos, tf.float64), (venue_id, tf.int32), (orig_cat_id, tf.int32)])), (y, tf.int32)])>


The preprocessing also works as intended.
Now, the model is trained and evaluated.
Logs are saved in a dedicated folder.

In [30]:
train_and_eval_model(vocab_size, n, federated_train_data, federated_val_data, federated_test_data, path='./log/central-test-run')

Round 1
	Train: loss=2.681, accuracy=0.232
	Validation: loss=6.289, accuracy=0.146
	Saving best model..
 
	writing..
Round 2
	Train: loss=4.786, accuracy=0.222
	Validation: loss=8.664, accuracy=0.058
 
	writing..
Round 3
	Train: loss=9.036, accuracy=0.066
	Validation: loss=4.726, accuracy=0.052
	Saving best model..
 
	writing..
Round 4
	Train: loss=4.255, accuracy=0.115
	Validation: loss=5.453, accuracy=0.174
 
	writing..
Round 5
	Train: loss=4.543, accuracy=0.198
	Validation: loss=4.398, accuracy=0.144
	Saving best model..
 
	writing..
Round 6
	Train: loss=3.615, accuracy=0.225
	Validation: loss=3.521, accuracy=0.045
	Saving best model..
 
	writing..
Round 7
	Train: loss=2.999, accuracy=0.183
	Validation: loss=3.720, accuracy=0.049
 
	writing..
Round 8
	Train: loss=3.246, accuracy=0.153
	Validation: loss=3.295, accuracy=0.094
	Saving best model..
 
	writing..
Round 9
	Train: loss=2.988, accuracy=0.191
	Validation: loss=3.353, accuracy=0.174
 
	writing..
Round 10
	Train: loss=2.841, ac

Adding more columns resulted in a worse model performance.
Many columns that were used can not be used to identify the next semantic location.
For example, lat/long, the original category id or the venue id.
The algorithm should not be able to learn from those features.
The semantic location does not correlate with physical locations in form of coordinates.
Using the more fine-grained location data also does not help the prediction instead it adds more features and complicates the model, thus reducing accuracy.
Integrating only those features that are known to have an impact on the selection of the next location is the best choice.